In [25]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [26]:
def func(x: list[float, float, float, float], step=-1):
    lam = x[3]
    
    if step == 3:
        return sum(el ** 2 for el in x[:-1])
    elif step in [0, 1, 2]:
        return x[step] * (1 + 2 * lam) + 1
    return 1/2 * sum(el ** 2 for el in x[:-1]) + sum(el for el in x[:-1]) + lam * (sum(el ** 2 for el in x[:-1]) - 3)


def len_vec(px: list[float, float, float, float]):
    # print(px)
    return np.sqrt(sum(el ** 2 for el in px))

In [27]:
# Градиентный спуск

In [28]:
def main_cycle(x: list[float, float, float, float], h: float) -> list[float, float, float, float]:
    return [pos - h * func(x, i) for i, pos in enumerate(x)]

In [29]:
eps = 0.001
begin_step = 0.1
h = begin_step
columns = ["iter", "h", "x", "y", "z", "l", "f(x, y, z, l)", "diff", "f`_x", "f`_y", "f`_z", "f'_l", "norm"]

res_columns = ["start_pos"] + columns

results_df = pd.DataFrame(columns=res_columns)
# df.loc[len(df.index)] = data

In [30]:
s_lambda = 0

for posx in range(-5, 5 + 1, 2):
    for posy in range(-5, 5 + 1, 2):
        for posz in range(-5, 5 + 1, 2):
            index = 0
        
            start_pos = [posx, posy, posz, s_lambda]
        
            vec = start_pos
            px = [func(vec, i) for i in range(4)]
            # print(px)
            norm = len_vec(px)
            prev_res = func(start_pos)
            diff = -1
            
            index += 1
            
            data = [index, h,
                    vec[0], vec[1], vec[2], vec[3],
                    prev_res, None, px[0], px[1], px[2], px[3],
                    norm]
            # print(len(data), len(columns))
            df = pd.DataFrame([data], columns=columns)
            
            while norm > eps:
                if diff > 0:
                    h /= 2
                if index > 50:
                    break
                vec = main_cycle(vec, h)
                vec = vec / np.linalg.norm(vec) * np.sqrt(3)
                res = func(vec)
                diff = res - prev_res
                prev_res = res
                px = [func(vec, i) for i in range(4)]
                norm = len_vec(px)
                # print(px, norm)

                index += 1
                data = [index, h,
                        vec[0], vec[1], vec[2], vec[3],
                        res, diff, px[0], px[1], px[2], px[3],
                        norm]
                # print(data)
                df.loc[len(df.index)] = data
            
            results_df.loc[len(results_df.index)] = [start_pos] + data
            df.to_csv(f'res/start_pos_{start_pos} precision_{eps} begin_step_{begin_step}.csv')

results_df    

,start_pos,iter,h,x,y,z,l,"f(x, y, z, l)",diff,f`_x,f`_y,f`_z,f'_l,norm
0,"[-5, -5, -5, 0]",51,1.000000e-01,-0.829868,-0.829868,-0.829868,-9.664151e-01,-0.553991,-5.746448e-11,1.774126,1.774126,1.774126,2.066042,3.702850
1,"[-5, -5, -3, 0]",51,1.000000e-01,-0.830590,-0.830590,-0.828421,-9.664153e-01,-0.553989,-6.117456e-07,1.774800,1.774800,1.772777,2.066041,3.702849
2,"[-5, -5, -1, 0]",51,1.000000e-01,-0.831489,-0.831489,-0.826614,-9.664163e-01,-0.553978,-3.091255e-06,1.775640,1.775640,1.771093,2.066040,3.702848
3,"[-5, -5, 1, 0]",51,1.000000e-01,-0.832499,-0.832499,-0.824577,-9.664183e-01,-0.553956,-8.162465e-06,1.776585,1.776585,1.769196,2.066036,3.702846
4,"[-5, -5, 3, 0]",51,1.000000e-01,-0.833491,-0.833491,-0.822567,-9.664213e-01,-0.553925,-1.552159e-05,1.777516,1.777516,1.767325,2.066030,3.702842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,"[5, 5, -3, 0]",51,2.869859e-43,1.127469,1.127469,-0.676481,-4.600503e-41,3.078457,0.000000e+00,2.127469,2.127469,0.323519,3.000000,4.261093
212,"[5, 5, -1, 0]",51,2.869859e-43,1.212678,1.212678,-0.242536,-4.573675e-41,3.682821,0.000000e+00,2.212678,2.212678,0.757464,3.000000,4.400641
213,"[5, 5, 1, 0]",51,2.869859e-43,1.212678,1.212678,0.242536,-4.573675e-41,4.167892,0.000000e+00,2.212678,2.212678,1.242536,3.000000,4.509521
214,"[5, 5, 3, 0]",51,2.869859e-43,1.127469,1.127469,0.676481,-4.600503e-41,4.431420,0.000000e+00,2.127469,2.127469,1.676481,3.000000,4.567586


In [31]:
# Настоящий результат: -1.16666666.....7
#(*) = (-1.333..., -1.666..., 0.666...)